In [1]:
! . /usr/local/bin/thisroot.sh
import ROOT
from ROOT import TFile,TH1,TH1F,TCanvas,TLine
from ROOT import RooFit

Welcome to JupyROOT 6.10/08


In [2]:
! ls ../rootfiles/X4140*.root

../rootfiles/X4140_MuMuKK.root                                                         ../rootfiles/X4140_MuMuKK_KRe_MuRef_EtaX1.1_NP3.5_Alpha99_CW5.2-5.55.root
../rootfiles/X4140_MuMuKK_KRe_MuMixed_NP3.5_Alpha99_CW5.2-5.55.root                    ../rootfiles/X4140_MuMuKK_KRe_MuRef_NP3.5_Alpha99_CW5.15-5.55.root
../rootfiles/X4140_MuMuKK_KRe_MuMixed_NP3_Alpha99.root                                 ../rootfiles/X4140_MuMuKK_KRe_MuRef_NP3.5_Alpha99_DM_CW5.15-5.55.root
../rootfiles/X4140_MuMuKK_KRe_MuRe_ChiCuts_NP3.0_Alpha99_CW5.15-5.55.root              ../rootfiles/X4140_MuMuKK_KRe_MuRef_SIDES_NP3.0_Alpha99_CW5.2-5.55.root
../rootfiles/X4140_MuMuKK_KRe_MuRe_NP3.0_Alpha99_CW5.15-5.55_GOODx5568.root            ../rootfiles/X4140_MuMuKK_KRe_MuRef_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55.root
../rootfiles/X4140_MuMuKK_KRe_MuRe_NP3.0_Alpha99_CW5.15-5.55_PtJPsi7_PtMu_4.root       ../rootfiles/X4140_MuMuKK_KRe_MuRef_Sidebands_NP3.5_Alpha99_CW5.15-5.55.root
../rootfiles/X4140_MuMuKK_KRe_MuRe_NP3.

In [3]:
#rootfile ="../rootfiles/X4140_MuMuKK_KRe_MuRe_NP3.0_Alpha99_CW5.15-5.55_PtJPsi7_PtMu_4_NOPHI.root"
rootfile ="../rootfiles/X4140_MuMuKK_KRe_MuRef_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55.root"
#histname = "Xcand_histo_hlt8_cw_nonprompt_cosalpha"
#histname = "Phi_hist_all_cw_all_cosalpha"
histname = "PhiMassHisto"
#histname = "Xcand_histo_DM_any_cw_nonprompt_cosalpha"
inputfile  = TFile(rootfile,"READ")
hist = inputfile.Get(histname)
c = TCanvas("canvas","canvas",1200,800) ;

In [4]:
from ROOT import RooRealVar,RooAbsPdf,RooChebychev,RooExponential,RooGaussian,RooAbsPdf,RooPlot,RooAddPdf,RooDataHist,RooArgSet,RooArgList
from ROOT import kGreen,kRed,kBlack,kBlue,kDashed,kDotted,kMagenta
from ROOT.RooFit import Components,LineColor,LineStyle,Name,Normalization,Layout,Format,Label,Parameters,Range

In [5]:
massmin = 1.020-0.030
massmax = 1.020+0.030
phimean = 1.020
massbins = (massmax - massmin)/hist.GetBinWidth(2)
mass = RooRealVar("mass","M(KK)[GeV]",massmin,massmax)
mass.setBins(int(massbins))

In [6]:
hist.GetXaxis().SetRangeUser(massmin,massmax)
hist.Draw()
plotname = "plots/phiplot_" + histname
c.SaveAs(plotname  + ".png")
c.SaveAs(plotname  + ".eps")
c.SaveAs(plotname  + ".pdf")

Info in <TCanvas::Print>: png file plots/phiplot_PhiMassHisto.png has been created
Info in <TCanvas::Print>: eps file plots/phiplot_PhiMassHisto.eps has been created
Info in <TCanvas::Print>: pdf file plots/phiplot_PhiMassHisto.pdf has been created


In [7]:
mean = RooRealVar("mean","mean of gaussian",phimean,phimean-0.005,phimean+0.005);
sigma = RooRealVar("sigma1","width of gaussian1",0.001,0.0005,0.05);

a0 = RooRealVar("a0","a0",0.001,-1.,1.)
a1 = RooRealVar("a1","a1",0.001,-0.5,0.5)
a2 = RooRealVar("a2","a2",-0.00001,-2.,2.)
a3 = RooRealVar("a3","a3",-0.000001,-0.1,0.1)
a4 = RooRealVar("a4","a4",0.0,-0.1,0.1)
a5 = RooRealVar("a5","a5",0.0,-0.025,0.05)
a6 = RooRealVar("a6","a6",0.0,-0.001,0.001)

aset = RooArgList(a0,a1,a2,a3,a4,a5)
sFrac = RooRealVar("sFrac","sFrac",0.5,0.,1.0)

In [8]:
cheb = RooChebychev("cheb","Background",mass,aset)
gauss = RooGaussian("gauss","gaussian PDF ",mass,mean,sigma)

tot = RooAddPdf("tot","g+cheb",gauss,cheb,sFrac)

In [9]:
masslist = RooArgList(mass)
dh = RooDataHist("dh","dh",masslist,hist)
numEvts = dh.sum(False)
print numEvts

3074160.0
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(dh): fit range of variable mass expanded to nearest bin boundaries: [0.99,1.05] --> [0.99,1.05]


In [10]:
tot.fitTo(dh)

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss,cheb)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           1.00000e-03  2.00000e-01   -1.00000e+00  1.00000e+00
     2 a1           1.00000e-03  1.00000e-01   -5.00000e-01  5.00000e-01
     3 a2          -1.00000e-05  4.00000e-01   -2.00000e+00  2.00000e+00
     4 a3          -1.00000e-06  2.00000e-02   -1.00000e-01  1.00000e-01
     5 a4           0.00000e+00  2.00000e-02   -1.00000e-01  1.00000e-01
     6 a5           0.00000e+00  7.50000e-03   -2.50000e-02  5.00000e-02
     7 mean         1.02000e+00  1.00000e-03    1.01500e+00  1.02500e+00
     8 sFrac        5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     9 sigma1       1.00000e-03  2.50000e-0

In [11]:
massFrame = mass.frame()
massFrame.SetTitle("Phi signal")
dh.plotOn(massFrame)
tot.plotOn(massFrame)
gauss.plotOn(massFrame,LineColor(kGreen),LineStyle(kDashed),Normalization((sFrac.getValV()*numEvts)/(numEvts)))
cheb.plotOn(massFrame,LineColor(kMagenta),LineStyle(kDotted),Normalization(((1.0-sFrac.getValV())*numEvts)/(numEvts)))
tot.paramOn(massFrame,Layout(0.60,0.99,0.75));
massFrame.Draw()

In [12]:
plotmax = hist.GetMaximum()*1.05
sidesigma = sigma.getValV()
leftlowside = -7.*sidesigma + mean.getValV()
leftupside = -5.*sidesigma + mean.getValV()
rightlowside = +5.*sidesigma + mean.getValV()
rightupside = +7.*sidesigma + mean.getValV()

signallow = -3.*sidesigma + mean.getValV()
signalup = +3.*sidesigma + mean.getValV()

print "Side Sigma"
print "%.5e" % sidesigma
print "Left Side"
print "%.5f - %.5f" % (leftlowside,leftupside)
print mean.getValV()

Side Sigma
2.35607e-03
Left Side
1.00308 - 1.00779
1.01956968216


In [13]:
linelow = TLine(leftlowside,0.0,leftlowside,plotmax)
linemidlow = TLine(leftupside,0.0,leftupside,plotmax)
linemidup = TLine(rightlowside,0.0,rightlowside,plotmax)
lineup = TLine(rightupside,0.0,rightupside,plotmax)

linesiglow = TLine(signallow,0.0,signallow,plotmax)
linesigup = TLine(signalup,0.0,signalup,plotmax)

linelow.SetLineColor(kRed)
linemidlow.SetLineColor(kRed)
linemidup.SetLineColor(kRed)
lineup.SetLineColor(kRed)

linesiglow.SetLineColor(kGreen)
linesigup.SetLineColor(kGreen) 

linelow.SetLineWidth(2)
linemidlow.SetLineWidth(2)
linemidup.SetLineWidth(2)
lineup.SetLineWidth(2)

linesiglow.SetLineWidth(2)
linesigup.SetLineWidth(2) 

linelow.Draw()
linemidlow.Draw()
linemidup.Draw()
lineup.Draw()

linesiglow.Draw()
linesigup.Draw()

In [14]:
plotname = "plots/phi_" + histname
c.SaveAs(plotname  + ".png")
c.SaveAs(plotname  + ".eps")
c.SaveAs(plotname  + ".pdf")

Info in <TCanvas::Print>: png file plots/phi_PhiMassHisto.png has been created
Info in <TCanvas::Print>: eps file plots/phi_PhiMassHisto.eps has been created
Info in <TCanvas::Print>: pdf file plots/phi_PhiMassHisto.pdf has been created


In [15]:
mass.setRange("signalrange",signallow,signalup)
mass.setRange("sideleftrange",leftlowside,leftupside)
mass.setRange("siderightrange",rightlowside,rightupside)
signalIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)),"signalrange")
leftsideIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)),"sideleftrange")
rightsideIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)),"siderightrange")


[#1] INFO:Eval -- RooRealVar::setRange(mass) new range named 'signalrange' created with bounds [1.0125,1.02664]
[#1] INFO:Eval -- RooRealVar::setRange(mass) new range named 'sideleftrange' created with bounds [1.00308,1.00779]
[#1] INFO:Eval -- RooRealVar::setRange(mass) new range named 'siderightrange' created with bounds [1.03135,1.03606]


In [16]:
totIntegralBkg= cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)))

In [17]:
sigBkgEvts = signalIntegralBkg/totIntegralBkg*((1.0-sFrac.getValV())*numEvts)
sidBkgEvts = (leftsideIntegralBkg+rightsideIntegralBkg)/totIntegralBkg*((1.0-sFrac.getValV())*numEvts)

In [18]:
print sigBkgEvts

776754.154227


In [19]:
print sidBkgEvts

469905.004566


In [20]:
ratio = sigBkgEvts/sidBkgEvts

In [21]:
inputfile.ls()

TFile**		../rootfiles/X4140_MuMuKK_KRe_MuRef_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55.root	
 TFile*		../rootfiles/X4140_MuMuKK_KRe_MuRef_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55.root	
  OBJ: TH1F	PhiMassHisto	PhiMassHisto : 0 at: 0x7fc049f3a8d0
  OBJ: RooPlot	frame_mass_7fc04a3aa360	Phi signal : 0 at: 0x7fc04a3aa360
  KEY: TH1F	X_Cand_Mass;1	X_Cand_Mass
  KEY: TH1F	X_Cand_Mass_NoMult;1	X_Cand_Mass_NoMult
  KEY: TH1F	PhiMassHisto;1	PhiMassHisto
  KEY: TH1F	B0_Cand_Mass_Sides;1	B0_Cand_Mass_Sides
  KEY: TH1F	B0_Cand_Mass_Sides_NoM;1	B0_Cand_Mass_Sides_NoM
  KEY: TH1F	B0_Cand_Mass;1	B0_Cand_Mass
  KEY: TH1F	B0_Cand_Mass_NoM;1	B0_Cand_Mass_NoM


In [22]:
signalB0 = inputfile.Get("B0_Cand_Mass_NoM")
sideB0 = inputfile.Get("B0_Cand_Mass_Sides_NoM")
notrebin = True

In [23]:
if notrebin:
    signalB0.Rebin(5)
    sideB0.Rebin(5)
    notrebin = False

In [24]:
sideB0.Scale(ratio)

In [25]:
cB0 = TCanvas("cB0","cB0",1200,800)
signalB0.SetFillColor(kBlue)
signalB0.SetFillStyle(3002)
sideB0.SetFillColor(kRed)
sideB0.SetFillStyle(3002)
signalB0.Draw()
sideB0.Draw("SAMEBar")
cB0.SaveAs("plots/" + signalB0.GetName() + "_sidebands.png")

Info in <TCanvas::Print>: png file plots/B0_Cand_Mass_NoM_sidebands.png has been created


In [26]:
b0SideSubtracted = signalB0.Clone()
b0SideSubtracted.Add(sideB0,-1.0)

True

In [27]:
b0SideSubtracted.SetFillColor(kGreen)
b0SideSubtracted.SetFillStyle(3002)
b0SideSubtracted.Draw("Bar")
cB0.SaveAs("plots/" + signalB0.GetName() + "_subtracted.png")

Info in <TCanvas::Print>: png file plots/B0_Cand_Mass_NoM_subtracted.png has been created
